In [28]:
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np
import operator
import pickle

import src.fusiontools as fusiontools
import src.scoring as scoring
import src.postprocess as postprocess

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
path_to_results = 'results/'

# Load best models

In [60]:
# Load best recency model
recency_rank_dic_path = 'ranks_test/recency_beta_7000_rec-all'
with open(recency_rank_dic_path, 'rb') as infile:
    recency_ranks = pickle.load(infile)
    
# Load best knn model
knn_rank_dic_path = 'ranks_test/knn-k-50-rec-all'
with open(knn_rank_dic_path, 'rb') as infile:
    knn_ranks = pickle.load(infile)

# Load best bayesian model
bayes_rank_dic_path = 'ranks_test/bayesian_results_0-35785'
with open(bayes_rank_dic_path, 'rb') as infile:
    bayes_ranks = pickle.load(infile)
    
# # Load best graph_through_words model
words_rank_dic_path = 'ranks_test/resjo_pickle_318'
with open(words_rank_dic_path, 'rb') as infile:
    words_ranks = pickle.load(infile)

# Load best graph_through_words model
centroids_rank_dic_path = 'ranks_test/centroids_dict_test'
with open(centroids_rank_dic_path, 'rb') as infile:
    centroids_ranks = pickle.load(infile)
    
# Load best word2vec model
word2vec_rank_dic_path = 'ranks_test/centroids_w2v_sw0_test'
with open(word2vec_rank_dic_path, 'rb') as infile:
    word2vec_ranks = pickle.load(infile)
    
# Load best twidf model
twidf_knn_rank_dic_path = 'ranks_test/twidf-knn-k-50-rec'
with open(twidf_knn_rank_dic_path, 'rb') as infile:
    twidf_knn_ranks = pickle.load(infile)
    
# Load best twidf sender model
twidf_knn_sender_rank_dic_path = 'ranks_test/twidf-knn-k-senders-50-rec'
with open(twidf_knn_sender_rank_dic_path, 'rb') as infile:
    twidf_knn_sender_ranks = pickle.load(infile)

# Discard tail of recipients

In [61]:
nb_kept_recips = 100
knn_ranks_cropped = fusiontools.keep_only_max_recips(knn_ranks, nb_kept_recips)
recency_ranks_cropped = fusiontools.keep_only_max_recips(recency_ranks, nb_kept_recips)
bayes_ranks_cropped = fusiontools.keep_only_max_recips(bayes_ranks, nb_kept_recips)
centroids_ranks_cropped = fusiontools.keep_only_max_recips(centroids_ranks, nb_kept_recips)
words_ranks_cropped = fusiontools.keep_only_max_recips(words_ranks, nb_kept_recips)
twidf_knn_sender_ranks_cropped = fusiontools.keep_only_max_recips(twidf_knn_sender_ranks, nb_kept_recips)

# Prepare models and weights

In [63]:
# models = [knn_ranks, recency_ranks]
# weights = [0.20716, 0.39316]

# models = [bayes_100, recency_ranks]
# weights = [0.34218, 0.39316]



models = [bayes_ranks_cropped, recency_ranks_cropped, knn_ranks_cropped]
weights = [0.35785, 0.39316, 0.20716]


# models = [bayes_ranks_cropped, recency_ranks_cropped, words_ranks_cropped]
# weights = [0.35785, 0.39316, 0.33785]

# models = [bayes_ranks_cropped, recency_ranks_cropped, words_ranks_cropped, knn_ranks_cropped]
# weights = [0.35785*0.6, 0.39316*0.5, 0.33785*0.6, 0.20716*0.2]

# models = [knn_ranks_cropped, recency_ranks_cropped, bayes_ranks_cropped, centroid_ranks_cropped, words_ranks_cropped]
#val 0.5, 1, 0.5, 1, 0.5,
# models = [bayes_ranks_cropped, centroids_ranks_cropped, recency_ranks_cropped, knn_ranks_cropped, twidf_knn_sender_ranks_cropped]
# weights = [0.35785, 0.24153, 0.39316, 0.20716, 0.33785, 0.32109]

#Fast check that dictionnaries are the right lenght
for model in models:
    print(len(model))

2362
2362
2362


In [50]:
for mid, recipients in twidf_knn_sender_ranks_cropped.items():
    print(len(recipients))


100
90
45
76
1
1
62
31
50
100
55
96
18
100
100
19
53
18
100
100
47
46
59
100
75
100
100
53
100
92
63
55
65
100
44
100
67
67
36
56
58
27
27
63
30
86
55
100
1
47
76
100
45
52
100
36
100
36
78
67
100
44
85
100
100
59
100
60
64
100
100
91
100
54
64
100
95
95
75
62
14
53
100
100
63
30
100
100
66
100
61
100
47
92
100
27
75
100
85
33
100
100
27
100
43
52
100
32
33
32
100
34
33
16
100
100
100
32
41
58
100
43
100
100
51
39
87
52
100
56
37
40
56
100
100
100
100
100
100
52
100
100
59
46
97
65
92
1
100
17
100
18
100
100
38
87
86
74
35
27
51
100
100
100
42
36
42
36
100
42
40
26
100
100
100
42
100
41
41
43
100
100
100
100
100
100
84
10
53
100
1
1
55
12
53
75
100
47
92
30
100
100
63
93
100
100
85
100
100
78
100
100
100
65
83
46
44
34
37
59
70
76
29
67
27
51
100
7
7
14
100
30
19
68
34
38
100
47
37
30
31
31
38
29
56
100
69
36
100
37
29
97
57
47
97
49
12
100
1
62
33
29
32
67
35
75
86
33
61
38
100
43
39
100
100
51
100
100
62
70
52
35
45
28
54
65
54
65
100
45
100
77
65
56
68
43
31
56
83
87
100
100
100
92


In [64]:
reranking_coeff = 7
fusion_dic = fusiontools.reciprocal_rerank(models, reranking_coeff,  weights=weights)
# print(fusion_dic)

            
                

In [65]:
# Retrieve email from bodies
emails_in_content_path = 'variables/emails_in_test'
with open(emails_in_content_path, 'rb') as infile:
    emails_in_content = pickle.load(infile)

In [81]:
manual_dic = {}
for mid, recipients in fusion_dic.items():
    if(mid in list(emails_in_content.keys())):
        found_mails = emails_in_content[mid] 
        recipients = [rec for rec in recipients if rec not in found_mails]
        manual_dic[mid] = found_mails + recipients
    else:
        manual_dic[mid] = recipients
        
    

In [82]:
postprocess.write_results_ranked(manual_dic, path_to_results, 'add-email-in-content-to-bayes-recency-knn-tfidf.csv')

In [27]:
postprocess.write_results_ranked(fusion_dic, path_to_results, 'fusion-weighted-reranking-15-bayes-recency-sender-twidf-all-tfidf-knn-centroids.csv')

In [83]:
189815 in emails_in_content.keys()

True